## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column.
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


,Attrition,Department
0,Yes,Sales
1,No,Research & Development
2,Yes,Research & Development
3,No,Research & Development
4,No,Research & Development


In [4]:
# Create a list of at least 10 column names to use as X data
selected_columns = [
    'Age', 'BusinessTravel', 'DistanceFromHome', 'Education',
    'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement',
    'JobLevel', 'JobSatisfaction', 'MaritalStatus'
]


# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes


Age                         int64
BusinessTravel             object
DistanceFromHome            int64
Education                   int64
EnvironmentSatisfaction     int64
HourlyRate                  int64
JobInvolvement              int64
JobLevel                    int64
JobSatisfaction             int64
MaritalStatus              object
dtype: object

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=42)


In [6]:
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)


In [7]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train_encoded)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train_encoded)
X_test_scaled = scaler.transform(X_test_encoded)

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
encoder = OneHotEncoder(sparse=False)


# Fit the encoder to the training data
encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_department_encoded = encoder.transform(y_train[['Department']])
y_test_department_encoded = encoder.transform(y_test[['Department']])




/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


## Create, Compile, and Train the Model

In [10]:
# Find the number of columns in the X training data
input_dim = X_train_scaled.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(input_dim))

# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(32, activation='relu')(shared_layer1)

In [11]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
department_output_layer = layers.Dense(y_train_department_encoded.shape[1], activation='softmax', name='department_output')(department_hidden_layer)


In [12]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_hidden_layer = layers.Dense(16, activation='relu')(shared_layer2)

# Create the output layer
attrition_output_layer = layers.Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_hidden_layer)


In [13]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 14)]                 0         []                            
                                                                                                  
 dense (Dense)               (None, 64)                   960       ['input_1[0][0]']             
                                                                                                  
 dense_1 (Dense)             (None, 32)                   2080      ['dense[0][0]']               
                                                                                                  
 dense_2 (Dense)             (None, 16)                   528       ['dense_1[0][0]']             
                                                                                              

In [19]:
# Train the model
history = model.fit(X_train_scaled,{'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded},
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_test_scaled, {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded}),
    verbose=1)

Epoch 1/100
35/35 [==============================] - 2s 7ms/step - loss: 1.4948 - department_output_loss: 0.9945 - attrition_output_loss: 0.5003 - department_output_accuracy: 0.5309 - attrition_output_accuracy: 0.8149 - val_loss: 1.2319 - val_department_output_loss: 0.8466 - val_attrition_output_loss: 0.3853 - val_department_output_accuracy: 0.6522 - val_attrition_output_accuracy: 0.8696
Epoch 2/100
35/35 [==============================] - 0s 5ms/step - loss: 1.2386 - department_output_loss: 0.8040 - attrition_output_loss: 0.4347 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8285 - val_loss: 1.1573 - val_department_output_loss: 0.7910 - val_attrition_output_loss: 0.3663 - val_department_output_accuracy: 0.6522 - val_attrition_output_accuracy: 0.8696
Epoch 3/100
35/35 [==============================] - 0s 7ms/step - loss: 1.1781 - department_output_loss: 0.7698 - attrition_output_loss: 0.4083 - department_output_accuracy: 0.6543 - attrition_output_accuracy: 0.8348 

In [16]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded},
    verbose=1
)

12/12 [==============================] - 1s 4ms/step - loss: 1.8906 - department_output_loss: 1.2691 - attrition_output_loss: 0.6215 - department_output_accuracy: 0.1277 - attrition_output_accuracy: 0.7310


In [22]:
# Print the accuracy for both department and attrition
# Calculate accuracy for the department output
department_pred = model.predict(X_test_scaled)[0]
department_accuracy = np.mean(np.argmax(department_pred, axis=1) == np.argmax(y_test_department_encoded, axis=1))

# Calculate accuracy for the attrition output
attrition_pred = model.predict(X_test_scaled)[1]
attrition_accuracy = np.mean(np.argmax(attrition_pred, axis=1) == np.argmax(y_test_attrition_encoded, axis=1))

print(f"Test Department Output Accuracy: {department_accuracy}")
print(f"Test Attrition Output Accuracy: {attrition_accuracy}")

12/12 [==============================] - 0s 3ms/step
Test Department Output Accuracy: 0.5733695652173914
Test Attrition Output Accuracy: 0.782608695652174


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No, accuracy isnt the best metric to use on this data due to class imbalances. Precision, recall, and F1-score provide better insights, especially for minority classes. These metrics factor in the true positive, false positive, and false negative rates, offering a more comprehension view of the effectiveness. 
2. Softmax was used for boht department_output and attrition_output. Softmax is used for multi-class classification in the department output to ensure the outputs sum to 1. Softmax is used for attrition_output to get probabilities of each class.
3. Some way to improve the model may include:
    1.Increased data training to improve model generalization; collect more data over time from more sources.
    2.Add or modify features to provide better input data; create new features like the ratio of total years.
    3.Implement regularization to prevent overfitting
    4.Incorporate hyperparameter tuning via adjusting learning rate, batch sizes, number of layers; try different values for these combinations to improve performance.
    5.Trial different neural networks architechtures or other machine learning models; for example CBBs and gradient boosting. 
